In [4]:
# Loadidng packages

from datetime import datetime
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import sys


from scipy.ndimage import imread
import matplotlib.pyplot as plt


In [5]:
# define fixed variables 

batch_size = 256
num_classes = 10
epochs = 10

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [6]:
# Loading image arrays

with np.load("notmnist_small.npz") as f:
        data = f['data']
        labels = f['labels']

x_train, x_test, y_train, y_test = train_test_split(data, labels)

In [7]:
# Convert from 0-255 to 0-1
x_train /= 255
x_test /= 255

In [8]:
# Resize from (m, 28, 28) to (m, 28, 28, 1), since Keras always assumes the 4th
# dimension in case of RGB images, even if we're just doing B/W images.
x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

In [ ]:
print("Found %s records. Splitting into %s training and %s test records." % (
    x_train.shape[0] + x_test.shape[0],
    x_train.shape[0],
    x_test.shape[0],
))


In [10]:
# Convert from numeric labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [11]:
# This is a simpler model that can easily be trained on a laptop
model = Sequential([
    Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax'),
    ])

In [12]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
print("Model Summary:")
print(model.summary())

In [14]:
# Declare tensorboard callback
tensorboard = TensorBoard(log_dir='./logs/%s' % datetime.now().strftime('%Y-%m-%d_%H:%M'),
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False)


In [ ]:
# Train model on training set
try:
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test),
              callbacks=[tensorboard])
except KeyboardInterrupt:
    print("\n\nCaught KeyboardInterrupt, stopping training!")


In [ ]:
# See how well it did on the test setr
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

In [18]:
# Save model for later usage

model.save('model.h5')

In [21]:
# visulization function
def visual(new_image):
    model = keras.models.load_model('model.h5')
    image = imread(new_image).reshape(1, 28, 28, 1)

    activation_maps = [
        (layer.name, K.function([model.input, K.learning_phase()], [layer.output])([image, 0.])[0])
        for layer in model.layers
    ]

    for layer_name, activation_map in activation_maps:
        print('Activation map for layer %s %s' % (layer_name, activation_map.shape))

        if len(activation_map.shape) == 4:
            # Convert from vertically-stacked images to side-by-side in a line
            letters = np.hstack(np.transpose(activation_map[0], (2, 0, 1)))

            # Rearrange those images into a square
            activations = np.vstack(np.split(letters, int(activation_map.shape[-1] ** 0.5), 1))

        elif len(activation_map.shape) == 2:
            # try to make it square as much as possible. we can skip some activations.
            activations = activation_map[0]
            num_activations = len(activations)
            if num_activations > 1024:  # too hard to display it on the screen.
                square_param = int(np.floor(np.sqrt(num_activations)))
                activations = activations[0: square_param * square_param]
                activations = np.reshape(activations, (square_param, square_param))
            else:
                activations = np.expand_dims(activations, axis=0)

        else:
            raise Exception('Can\'t deal with shape %s' % (activation.shape,))

        plt.imshow(activations, interpolation='None', cmap='binary_r')
        plt.axis('off')
        plt.show()


In [ ]:
# testing visualization

visual("notMNIST_small/G/MTAuMTUgU2F0dXJkYXkgTmlnaHQgQlJLLnR0Zg==.png")